# Analys av klimatpåverkande utsläpp från svenska anläggningar

av [Anna Norberg](www.twitter.com/annanorberg)  

Grunddata kommer från det svenska [utsläppsregistret](http://utslappisiffror.naturvardsverket.se/) "Utsläpp i siffror" hos Naturvårdsverket dit företagen själva rapporterar in från sina miljörapporter. Resultatet från analysen användes som underlag till en artikel som skickades ut till Nyhetsbyrån Sirens kunder den 12 december 2019.  

In [41]:
options(scipen=999, warn=-1)

library(dplyr)
library(here)
library(readr)
library(openxlsx)
library(reshape2)
library(readxl)
library(tidyr)

**Läs in data.** 

Från "Utsläpp i siffror" fås tre separata Excelfiler, en för varje ämne och alla anläggningar. Potentiellt borde en kunna ladda ner bara en Excelfil med alla anläggningar och alla valda ämnen, men det lyckades inte.

In [42]:
raw_CH4 <- read_excel(here("raw_data", "CH4.xlsx"), skip=5)
raw_CO2fossilt <- read_excel(here("raw_data", "CO2fossilt.xlsx"), skip=5)
raw_N2O <- read_excel(here("raw_data", "N2O.xlsx"), skip=5)

Så här ser grundfilen ut.

In [43]:
head(raw_CH4)

Anläggning,Verksamhet enligt PRTR,Verksamhet enligt MPF,Län,Kommun,Vattendistrikt,Organisation,Organisationsnummer,Fastighetsbeteckning,Tillsynsmyndighet,Miljöledningssystem,År,Ämne,Utsläpp till luft (kg),Metod,Summa utsläpp till luft för länet under året,Summa utsläpp till vatten för länet under året,Summa utsläpp till reningsverk för länet under året,Summa alla utsläpp till luft under året
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Annelunds Avfallsupplag,5.(d) Avfallsdeponier (utom deponier för inert avfall och deponier som stängdes definitivt före den 16 juli 2001 eller för vilka den efterbehandlingsfasenenligt artikel 13 - 1999/31/EG av den 26 april 1999 löpt ut),90.300-i Deponering,Uppsala län,Enköping,Norra Östersjön,VafabMiljö kommunalförbund,222000-3129,SNEDEN S:1,Enköpings kommun,ISO 14001,2018,CH4,202000,C,220200,0,0,12152750
Atleverket,5.(d) Avfallsdeponier (utom deponier för inert avfall och deponier som stängdes definitivt före den 16 juli 2001 eller för vilka den efterbehandlingsfasenenligt artikel 13 - 1999/31/EG av den 26 april 1999 löpt ut),90.290-i Deponering,Örebro län,Örebro,Norra Östersjön,"Tekniska förvaltningen, Örebro kommun",212000-1967,ATTERSTA 7:8,Örebro kommun,Miljödiplomering (certifierad 2018),2018,CH4,1446190,C,1467974,0,0,12152750
Beleverket i Hässleholm,1.(c) Värmekraftverk och andra förbränningsanläggningar,90.210-i Förbränning,Skåne län,Hässleholm,Södra Östersjön,Hässleholm Miljö AB,556555-0349,HÄSSLEHOLM 87:10,Hässleholms kommun,NA,2018,CH4,175,M,893232,0,0,12152750
BillerudKorsnäs Karlsborgs AB,6.(a) Anläggningar för framställning av pappersmassa av trä eller liknande fibrösa material,"21.10-i Massa, papper och pappersvaror",Norrbottens län,Kalix,Bottenviken,BillerudKorsnäs Sweden AB Karlsborg Bruk,556876-2974,Karlsborg 3:1,Länsstyrelsen i Norrbottens län,ISO 14001,2018,CH4,18093,C,72589,0,0,12152750
BillerudKorsnäs Rockhammar AB,6.(a) Anläggningar för framställning av pappersmassa av trä eller liknande fibrösa material,"21.10-i Massa, papper och pappersvaror",Örebro län,Lindesberg,Norra Östersjön,BillerudKorsnäs Rockhammar AB,556761-2436,ROCKHAMMAR 1:3,Länsstyrelsen i Örebro län,ISO 14001:2004,2018,CH4,835,C,1467974,0,0,12152750
"BillerudKorsnäs Skog & Industri AB, Frövi",6.(a) Anläggningar för framställning av pappersmassa av trä eller liknande fibrösa material,"21.10-i Massa, papper och pappersvaror",Örebro län,Lindesberg,Norra Östersjön,BillerudKorsnäs Skog & Industri AB,556023-8338,FRÖVI 1:46,Länsstyrelsen i Örebro län,ISO 14001:2004,2018,CH4,20949,C,1467974,0,0,12152750


**Kombinera till ett dataset. Välj ut relevanta kolumner.**

In [44]:
CH4 <- raw_CH4 %>% 
  rename(CH4=`Utsläpp till luft (kg)`) %>% 
  select(Län, Kommun, Anläggning, Fastighetsbeteckning, Organisationsnummer, År, CH4)

CO2 <- raw_CO2fossilt %>% 
  rename(CO2=`Utsläpp till luft (kg)`) %>% 
  select(Län, Kommun, Anläggning, Fastighetsbeteckning, Organisationsnummer, År, CO2)

N2O <- raw_N2O %>% 
  rename(N2O=`Utsläpp till luft (kg)`) %>% 
  select(Län, Kommun, Anläggning, Fastighetsbeteckning, Organisationsnummer, År, N2O)

df <- full_join(CH4, CO2, by=c("Län", "Kommun", "Anläggning", "Fastighetsbeteckning", "Organisationsnummer", "År"))
df <- full_join(df, N2O, by=c("Län", "Kommun", "Anläggning", "Fastighetsbeteckning", "Organisationsnummer", "År"))

In [45]:
head(df)

Län,Kommun,Anläggning,Fastighetsbeteckning,Organisationsnummer,År,CH4,CO2,N2O
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Uppsala län,Enköping,Annelunds Avfallsupplag,SNEDEN S:1,222000-3129,2018,202000,NA,NA
Örebro län,Örebro,Atleverket,ATTERSTA 7:8,212000-1967,2018,1446190,NA,NA
Skåne län,Hässleholm,Beleverket i Hässleholm,HÄSSLEHOLM 87:10,556555-0349,2018,175,20487000,562
Norrbottens län,Kalix,BillerudKorsnäs Karlsborgs AB,Karlsborg 3:1,556876-2974,2018,18093,7298000,16589
Örebro län,Lindesberg,BillerudKorsnäs Rockhammar AB,ROCKHAMMAR 1:3,556761-2436,2018,835,2326250,835
Örebro län,Lindesberg,"BillerudKorsnäs Skog & Industri AB, Frövi",FRÖVI 1:46,556023-8338,2018,20949,15947492,10778


**Städa anläggningsnamn.**

En hel del företagsnamn är inte exakt demsamma från år till år. I en del fall har företaget bytt namn, men i andra fall är det skrivfel eller bara ett extra mellanslag i namnet. För att kunna göra en sammanställning i R behöver vi städa lite.

In [46]:
df$Anläggning[df$Anläggning=="Johannes Biokraftvärmeverk" |
                df$Anläggning=="Johannes biovärmeverk"|
                df$Anläggning=="Johannes Biovärmeverk" | 
                df$Anläggning=="Johannes kraftvärmeverk" ] <- "Johannes Kraftvärmeverk"
df$Anläggning[df$Anläggning=="DÅVA KRAFTVÄRMEVERK" ] <- "Dåva kraftvärmevärk"
df$Anläggning[df$Anläggning=="AB Nynäs petroleum" |
                df$Anläggning=="Nynas AB, Raffinaderiet i Göteborg"] <- "Nynas AB"
df$Anläggning[df$Anläggning=="SOFIELUNDS ÅTERVINNINGSANLÄGGNING"] <- "SOFIELUNDS ÅTERVINNINGSANLÄGGN"
df$Anläggning[df$Anläggning=="Granskärs Kraftvärmeverk"] <- "Granskärs kraftvärmeverk"
df$Anläggning[df$Anläggning=="Avfallsförbränningsanläggn"] <- "Boländeranläggningarna, Avfallsförbränningsanläggn"
df$Anläggning[df$Anläggning=="HÖGBYTORPSAVFALLSANLÄGGNING"] <- "HÖGBYTORPS AVFALLSANLÄGGNING"
df$Anläggning[df$Anläggning=="Shell Raffinaderi AB" | 
                df$Anläggning=="St1 Refinery  AB"] <- "St1 Refinery AB"
df$Anläggning[df$Anläggning=="Kronogårdens Värmeverk"] <- "Värmecentral Kronogården"
df$Anläggning[df$Anläggning=="Katrineholm Energi AB, PC Väst"] <- "Kraftvärmeverket i Katrineholm"
df$Anläggning[df$Anläggning=="Norsaverkets avfallsförbränningsanläggning"] <- "Norsaverkets avfallsförbrännin"
df$Anläggning[df$Anläggning=="M-real Sverige AB, Husums fabr"|
                df$Anläggning=="M-real Sverige AB, Husums fabrik"] <- "Metsä Board Sverige AB, Husums fabr"
df$Anläggning[df$Anläggning=="NSR återvinningsanläggning Helsingborg"] <- "NSR återvinningsanläggning Hel"
df$Anläggning[df$Anläggning=="Fiskeby Bruk"] <- "Fiskeby bruk"
df$Anläggning[df$Anläggning=="Avfallskraftvärmeverket Torsvik"|
                df$Anläggning=="Kraftvärmeverket Torsvik"] <- "Kraftvärmeverket Torsvik, KVVT1 (avfall) och KVVT2 (flis)"
df$Anläggning[df$Anläggning=="Korsnäs AB, Frövi"|
                df$Anläggning=="Korsnäs Frövi AB"] <- "BillerudKorsnäs Skog & Industri AB, Frövi"
df$Anläggning[df$Anläggning=="Stallbacka Kraftstation"|
                df$Anläggning=="Stallbacka Kraftstation Block 1 & 2"] <- "Stallbacka Kraftstation Block"
df$Anläggning[df$Anläggning=="Lextorps Värmeverk"] <- "Värmecentral Lextorp"
df$Anläggning[df$Anläggning=="CELANESE EMULSIONS NORDEN AB"] <- "CELANESE Production Sweden AB"
df$Anläggning[df$Anläggning=="Iggesund Paperboard AB, Iggesu"|
                df$Anläggning=="Iggesund Paperboard AB, Iggesund Bruk"] <- "Iggesunds Bruk"
df$Anläggning[df$Anläggning=="SMA Svenska Mineral AB, RÄTTVIKS KALKVERK"] <- "RÄTTVIKS KALKVERK"
df$Anläggning[df$Anläggning=="HEDENSBYNS KRAFTVÄRMEVERK"] <- "Hedensbyns kraftvärmeverk"
df$Anläggning[df$Anläggning=="E.ON Värmekraft Sverige AB, Halmstad"|
                df$Anläggning=="E.ON Värmekraft Sverige AB, Halmstadsverket"|
                df$Anläggning=="Sydkraft Thermal Power AB, Halmstadsverket"|
                df$Anläggning=="E.ON Värmekraft Sverige AB, Halmstadverket"] <- "Halmstadsverket"
df$Anläggning[df$Anläggning=="Rödjorna - avfallsupplag"|
                df$Anläggning=="Rödjorna - återvinningsanläggning"|
                df$Anläggning=="Rödjorna, SUEZ Recycling AB"] <- "SUEZ Recycling AB, Rödjorna, Härlingstorp 4:1 och 5:1"
df$Anläggning[df$Anläggning=="SSAB EMEA i Borlänge"|
                df$Anläggning=="SSAB TUNNPLÅT AB"|
                df$Anläggning=="SSAB Tunnplåt i Borlänge"] <- "SSAB EMEA AB"
df$Anläggning[df$Anläggning=="AB Sandvik Materials Technolog"|
                df$Anläggning=="AB Sandvik Materials Technology"] <- "Sandvik AB"
df$Anläggning[df$Anläggning=="NYNAS AB, Raffinaderiet i Nynäshamn"] <- "NYNAS AB, Oljeraffinaderiet i Nynäshamn"
df$Anläggning[df$Anläggning=="IGELSTAVERKET"] <- "Igelsta värmeverk"
df$Anläggning[df$Anläggning=="Billerud Karlsborgs AB"] <- "BillerudKorsnäs Karlsborgs AB"
df$Anläggning[df$Anläggning=="KOVIKS ÅTERVINNINGSANLÄGGNING"] <- "KOVIKS ÅTERVINNINGSANLÄGGNING, SUEZ Recycling AB"
df$Anläggning[df$Anläggning=="INEOS NOVA SWEDEN AB"] <- "STYROLUTION SWEDEN AB"
df$Anläggning[df$Anläggning=="Ortvikens pappersbruk"] <- "SCA Ortviken"
df$Anläggning[df$Anläggning=="Sundsvall Energi AB, Korstaver"|
                df$Anläggning=="Sundsvall Energi AB, Korstaverket"] <- "Sundsvall Energi AB, Korstaverk"
df$Anläggning[df$Anläggning=="Kraftvärmeverket & Hetvattencentralen Vattumannen"] <- "Värmeverket Vattumannen"
df$Anläggning[df$Anläggning=="STORA ENSO KVARNSVEDEN AB"] <- "Stora Enso Paper AB, Kvarnsveden Mill"
df$Anläggning[df$Anläggning=="Ragn-Sells Heljestorp AB"] <- "Ragn-Sells Avfallsbehandling AB"
df$Anläggning[df$Anläggning=="Karlshamn Kraft AB"] <- "Karlshamnsverket"
df$Anläggning[df$Anläggning=="Linneberga avfallsupplag"] <- "Linneberga avfallsanläggning"
df$Anläggning[df$Anläggning=="Billerud Skärblacka AB"] <- "BillerudKorsnäs Sweden AB Skärblacka Bruk"
df$Anläggning[df$Anläggning=="AMCOR FLEXIBLES LUND AB"|
                df$Anläggning=="FLEXTRUS  AB"] <- "FLEXTRUS AB"
df$Anläggning[df$Anläggning=="SKOGSBACKA KRAFTVÄRMEVERK"] <- "Skogsbacka kraftvärmeverk"
df$Anläggning[df$Anläggning=="Kalkproduktion AB"] <- "Kalkproduktion Storugns AB"
df$Anläggning[df$Anläggning=="Lövängsverket (f d Kärnsjukhus"] <- "Lövängsverket (f d Kärnsjukhus)"
df$Anläggning[df$Anläggning=="Pappersbruket i Kisa"] <- "Sofidel Sweden AB"
df$Anläggning[df$Anläggning=="Sundsvall Energi AB, Nackstave"] <- "Sundsvall Energi AB, Nackstaverket"
df$Anläggning[df$Anläggning=="Ekokem AB, Norrtorp"] <- "Fortum Waste Solutions AB, Norrtorp"
df$Anläggning[df$Anläggning=="STORA ENSO NYMÖLLA AB"] <- "STORA ENSO PAPER AB"
df$Anläggning[df$Anläggning=="SCA Packaging Obbola AB"] <- "SCA Obbola AB"
df$Anläggning[df$Anläggning=="HÖGDALA AVFALLSANLÄGGNING"] <- "HÖGDALA AVFALLSANLÄGGNING, SUEZ Recycling AB"
df$Anläggning[df$Anläggning=="AarhusKarlshamn Sweden AB"] <- "AAK Sweden AB"
df$Anläggning[df$Anläggning=="Munksjö Aspa Bruk AB"] <- "Ahlstrom-Munksjö Aspa Bruk AB"
df$Anläggning[df$Anläggning=="Eka Chemicals AB, Albyfabriker"|
                df$Anläggning=="Eka Chemicals AB, Albyfabrikerna"] <- "Akzo Nobel Pulp and Performance, Albyfabriker"
df$Anläggning[df$Anläggning=="Perstorp Specialty Chemicals AB, Di-TMP-fabriken"] <- "Di-TMP-fabriken"
df$Anläggning[df$Anläggning=="Perstorp Specialty Chemicals AB, Formalinfabriken"] <- "Formalinfabriken"
df$Anläggning[df$Anläggning=="Perstorp Specialty Chemicals AB, Myrsyralinjen"] <- "Myrsyralinjen"
df$Anläggning[df$Anläggning=="Perstorp Specialty Chemicals AB, Neo-fabriken"] <- "Neo-fabriken"
df$Anläggning[df$Anläggning=="Perstorp Specialty Chemicals AB, Pentafabriken"] <- "Pentafabriken"
df$Anläggning[df$Anläggning=="Perstorp Specialty Chemicals AB, TMP-fabriken"] <- "TMP-fabriken"
df$Anläggning[df$Anläggning=="Perstorp Specialty Chemicals AB, Ångcentralen"] <- "Ångcentralen"
df$Anläggning[df$Anläggning=="RAYTOR COMPOUNDS AB"|
                df$Anläggning=="Chemiplastica AB - f d RAYTOR COMPOUNDS AB"|
                df$Anläggning=="CHEMIPLASTICA AB"] <- "BI-QEM AB"
df$Anläggning[df$Anläggning=="Perstorp Specialty Chemicals AB, Metalloxidfabrik"|
                df$Anläggning=="Formox AB, Metalloxidfabrik"] <- "FORMOX AB"
df$Anläggning[df$Anläggning=="Chemiplastica Resins AB"] <- "BI-QEM Resins AB"
df$Anläggning[df$Anläggning=="KVV-Åkerslund"] <- "Åkerslund"
df$Anläggning[df$Anläggning=="Styron Sverige AB"] <- "Trinseo Sverige AB"
df$Anläggning[df$Anläggning=="Rockhammars Bruk AB"|
                df$Anläggning=="Korsnäs Rockhammar AB"] <- "BillerudKorsnäs Rockhammar AB"
df$Anläggning[df$Anläggning=="Ryaverket"] <- "Gryaab AB Ryaverket"
df$Anläggning[df$Anläggning=="Boliden Mineral AB - Aitikgruv"] <- "Boliden Mineral AB - Aitikgruvan"
df$Anläggning[df$Anläggning=="Sysavs avfallsförbränningsanlä"] <- "Sysavs avfallsförbränningsanläggning"
df$Anläggning[df$Anläggning=="RÖNNSKÄRSVERKEN"] <- "Rönnskärsverken"
df$Anläggning[df$Anläggning=="Heden kraftvärmeverk (se 1780-1118)"] <- "Heden kraftvärmeverk"
df$Anläggning[df$Anläggning=="INEOS Sverige"|
                df$Anläggning=="INEOS Sverige AB"] <- "Inovyn Sverige AB"
df$Anläggning[df$Anläggning=="UTANSJÖ BRUK AB"] <- "Utansjö Bruk AB"
df$Anläggning[df$Anläggning=="SSAB ÉMEA AB - Oxelösund"|
                df$Anläggning=="SSAB Oxelösund AB"] <- "SSAB EMEA AB Oxelösund"
df$Anläggning[df$Anläggning=="Ahlstrom Ställdalen AB"] <- "Ahlstrom-Munksjö Ställdalen AB"
df$Anläggning[df$Anläggning=="Procordia Food AB Eslöv"] <- "Orkla Foods Sverige AB"
df$Anläggning[df$Anläggning=="Höganäs Sverige AB"] <- "Höganäs Sweden AB"
df$Anläggning[df$Anläggning=="LKAB - Svappavaaragruvan"] <- "LKAB - Svappavaaragruvan Leveäniemi"
df$Anläggning[df$Anläggning=="SSAB Tunnplåt AB"] <- "SSAB EMEA AB i Luleå"
df$Anläggning[df$Anläggning=="Västermalmsverket / Falu Kraft AB"] <- "Västermalmsverket / Falu Energi & Vatten AB"
df$Anläggning[df$Anläggning=="Preemraff Göteborg"|
                df$Anläggning=="Preem Petroleum AB Preemraff Göteborg"|
                df$Anläggning=="Preem AB Preemraff Göteborg"|
                df$Anläggning=="Preem AB (publ), Preemraff Göteborg"] <- "Preem AB Preemraff Göteborg"
df$Anläggning[df$Anläggning=="Värmeverket Södra Vakten 17"] <- "Värmeverket Södra Vakten 17/18"
df$Anläggning[df$Anläggning=="GENETA PANNCENTRAL"] <- "Geneta panncentral"
df$Anläggning[df$Anläggning=="VANKIVA AVFALLSANLÄGGNING"|
                df$Anläggning=="Hässleholms Kretsloppscenter, HKC"] <- "Hässleholms Kretsloppscenter"
df$Anläggning[df$Anläggning=="Visby Slakteri"|
                df$Anläggning=="Scan AB Visby"|
                df$Anläggning=="Scan AB - Visby"|
                df$Anläggning=="Gotlands Slagtreri AB"] <- "Gotlands Slagteri AB"
df$Anläggning[df$Anläggning=="Avfallskraftvärmeverket Kristi"|
                df$Anläggning=="Avfallskraftvärmeverket Kristinehed"|
                df$Anläggning=="Avfallsvärmeverket"] <- "Kristinehedsverket"
df$Anläggning[df$Anläggning=="Timboholm, Skövde Värmeverk"] <- "Timboholm, Skövde Värmeverk, Block 3"
df$Anläggning[df$Anläggning=="Stora Enso Hylte AB"] <- "Stora Enso Paper AB"
df$Anläggning[df$Anläggning=="E ON Värme Sverige AB, Åbyverket"|
                df$Anläggning=="E.ON Värme Sverige AB, Åbyverk"] <- "E.ON Värme Sverige AB, Åbyverket"
df$Anläggning[df$Anläggning=="ABB AB, Plast"] <- "ABB AB, Composite"
df$Anläggning[df$Anläggning=="Kraftvärmeverket Övik Energi A"] <- "Kraftvärmeverket Övik Energi AB"
df$Anläggning[df$Anläggning=="SCA Östrand"|
                df$Anläggning=="Östrands massafabrik"] <- "SCA Östrands massafabrik"

**Räkna om till koldioxidekvivalenter och summera per anläggning.**

Utsläppen av metan och dikväveoxid räknas om till koldioxidekvivalenter genom att multiplicera med omräkningsvärde 25 för metan och 298 för dikväveoxid. Här går det att läsa mer om omräkningsvärdena: https://www.naturvardsverket.se/Stod-i-miljoarbetet/Vagledningar/Luft-och-klimat/Berakna-dina-klimatutslapp/Sammanstall-totala-utslapp-av-vaxthusgaser-i-koldioxidekvivalenter/

Från början var tanken att inkludera även fluorerade kolväten (HFC), perfluorkolväten (PFC) och svavelhexafluorid (SF6), men det gick inte att hitta omräkningsvärden för hur ämnena redovisas i utsläppsregistret. Därför valde vi att fokusera på metan (CH4), dikväveoxid (N2O) och fossilt koldioxid (CO2), som är de tre största växthusgaserna.

In [47]:
df_ekv <- df %>% rowwise() %>% 
  mutate(Koldioxidekvivalenter=sum((CH4*25/1000), (CO2/1000), (N2O*298/1000), na.rm=TRUE)) %>% 
  group_by(Län, Kommun, Anläggning, Fastighetsbeteckning, Organisationsnummer, År) %>% 
  summarize("Utsläpp (ton CO2-ekv)"=sum(Koldioxidekvivalenter, na.rm=TRUE))

In [48]:
anlaggning_perar <- df_ekv  %>% spread(År, `Utsläpp (ton CO2-ekv)`)

In [49]:
head(anlaggning_perar)

Län,Kommun,Anläggning,Fastighetsbeteckning,Organisationsnummer,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Blekinge län,Karlshamn,AAK Sweden AB,OLJEFABRIKEN 2 (SOCKERBRUKET 19),556478-1796,NA,NA,6227.00,5709.00,4885.0,4679.00,4579.00,4366.00,8088.00,7375.00,6394.792,6091.179
Blekinge län,Karlshamn,Gasturbin Karlshamn,KÖLÖ 2,556112-5666,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,223.400,250.492
Blekinge län,Karlshamn,Karlshamnsverket,KÖLÖ 1 (KARLSHAMN 8:18-20),556112-5666,NA,80224.0,97417.00,152208.00,20207.0,26000.00,7829.00,1725.00,2678.00,8999.00,4748.500,5290.000
Blekinge län,Karlshamn,Södra Cell Mörrum,ELLEHOLM 35:1 (RÄVABYGGET 4:1),729500-3789,63915.6,72339.2,62405.92,76340.57,72213.7,73837.46,63810.18,35196.31,34150.56,22997.19,15651.266,41488.700
Blekinge län,Ronneby,Cascades Djupafors AB,"Häggatorp 2:24, 2:160 , 10:1 (Sänksjödal 1:1)",556344-8710,1662.0,1773.0,2742.00,3586.00,2621.0,1626.00,1403.00,NA,NA,NA,NA,NA
Dalarnas län,Avesta,"Outokumpu Stainless AB, Avesta",AVESTA 5:1,556001-8748,NA,NA,100892.00,113020.00,126131.0,111531.00,104745.00,105566.00,111826.00,129759.00,138972.000,134578.380


**Sortera anläggningarna länsvis och från högst till lägst utsläpp 2018.**

In [50]:
topplista <- df_ekv %>% filter(År==2018) %>% 
  arrange(desc(`Utsläpp (ton CO2-ekv)`)) %>% 
  select(-År)

lansvis <- arrange(topplista, Län, desc(`Utsläpp (ton CO2-ekv)`))

I topplistan kan vi se vilka anläggningar i Sverige som har de största klimatpåverkande utsläppen.

In [51]:
head(topplista, 10)

Län,Kommun,Anläggning,Fastighetsbeteckning,Organisationsnummer,Utsläpp (ton CO2-ekv)
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
Norrbottens län,Luleå,Luleå kraftvärmeverk LUKAB,Svartön 18:10,556195-0576,2120995.6
Västra Götalands län,Lysekil,"Preemraff, Lysekil","SJÖBOL 2:1 (ASPEDALEN 2:9, SJÖBOL 2:5)",556072-6977,1625082.0
Gotlands län,Gotland,"Cementa AB, Slitefabriken","OTHEM FILE 1:13 (OTHEM KLINTS 1:10, OTHEM NÄRS 1:180, OTHEM YTINGS 1:29, OTHEM ÖSTERBY 1:108, OTHEM ÖSTERBY 1:214)",556013-5864,1533816.0
Södermanlands län,Oxelösund,SSAB EMEA AB Oxelösund,"STORA HUMMELVIK 1:8 (ASPA 2:3, BRANNÄS 1:2, OXELÖ 7:37)",556313-7933,1462246.0
Norrbottens län,Luleå,SSAB EMEA AB i Luleå,"Svartön 18:11 (Svartön 18:19, Svartöstaden 13:36)",556313-7933,1058000.0
Västra Götalands län,Stenungsund,Borealis Krackeranl.,STENUNG 17:6,556078-6633,636536.0
Västra Götalands län,Göteborg,St1 Refinery AB,"BRÄCKE 42:1 (RÖDJAN 727:21, SYRHÅLA 765:193)",556050-2378,546914.0
Västra Götalands län,Göteborg,Preem AB Preemraff Göteborg,"SYRHÅLA 2:1 (ARENDAL 764:230, SYRHÅLA 2:2, SYRHÅLA 765:253)",556072-6977,540900.0
Stockholms län,Stockholm,VÄRTAVERKET,NIMROD 7,556016-9095,529438.4


Sorterar vi datan länsvis och i storleksordning efter utsläpp, får vi också en topplista för varje län.

In [52]:
head(lansvis)

Län,Kommun,Anläggning,Fastighetsbeteckning,Organisationsnummer,Utsläpp (ton CO2-ekv)
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
Blekinge län,Karlshamn,Södra Cell Mörrum,ELLEHOLM 35:1 (RÄVABYGGET 4:1),729500-3789,41488.700
Blekinge län,Karlshamn,AAK Sweden AB,OLJEFABRIKEN 2 (SOCKERBRUKET 19),556478-1796,6091.179
Blekinge län,Karlshamn,Karlshamnsverket,KÖLÖ 1 (KARLSHAMN 8:18-20),556112-5666,5290.000
Blekinge län,Karlshamn,Gasturbin Karlshamn,KÖLÖ 2,556112-5666,250.492
Dalarnas län,Borlänge,SSAB EMEA AB,JÄRNVERKET 1,556313-7933,241000.000
Dalarnas län,Avesta,"Outokumpu Stainless AB, Avesta",AVESTA 5:1,556001-8748,134578.380
